In [37]:
# Import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from math import radians, cos, sin, asin, sqrt
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import re
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor


In [38]:
# Load the data
train_data = pd.read_csv('data/X_train.csv')
Y_train = pd.read_csv('data/y_train.csv')
test_data = pd.read_csv('data/X_test.csv')

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19436\3271525685.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('data/X_train.csv')


In [39]:

# Chuyen ki tu khong xac dinh bang so "0" hoặc 2
train_data['constructionTime'].replace({'未知': 0}, inplace=True)
test_data['constructionTime'].replace({'未知': 0}, inplace=True)
train_data['livingRoom'].replace({'#NAME?': 2}, inplace=True)
test_data['livingRoom'].replace({'#NAME?': 2}, inplace=True)

# Cat 2 ky tu cuoi va gan gia tri moi vao cot floor
floor_col_train = train_data.floor.apply(lambda x: str(x)[-2:])
floor_col_test = test_data.floor.apply(lambda x: str(x)[-2:])

train_data['floor'] = floor_col_train
test_data['floor'] = floor_col_test

# We will find distance agnaist each lat and lng from Beijing (lat:39.916668,lon:116.383331)
def distance(lat2, lon2,lat1=39.916668,lon1=116.383331): 
      
    # The math module contains a function named 
    # radians which converts from degrees to radians. 
    lon1 = radians(lon1) 
    lon2 = radians(lon2) 
    lat1 = radians(lat1) 
    lat2 = radians(lat2) 
       
    # Haversine formula  
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  
    c = 2 * asin(sqrt(a))  
     
    # Radius of earth in kilometers. Use 3956 for miles 
    r = 6371
       
    # calculate the result 
    return(c * r) 

train_data['distance'] = train_data.apply(lambda x: distance (x['Lat'],x['Lng']),axis=1)
train_data['constructionTime'] = train_data['constructionTime'].astype(int)
train_data['building_age'] = 2024 - train_data['constructionTime']

test_data['distance'] = test_data.apply(lambda x: distance (x['Lat'],x['Lng']),axis=1)
test_data['constructionTime'] = test_data['constructionTime'].astype(int)
test_data['building_age'] = 2024 - test_data['constructionTime']
# Converting 'buildingType' feature to object or string type
train_data['buildingType'].replace(1,'Tower',inplace=True)
train_data['buildingType'].replace(2,'Bungalow',inplace=True)
train_data['buildingType'].replace(3,'Tower and Plate',inplace=True)
train_data['buildingType'].replace(4,'Plate',inplace=True)

test_data['buildingType'].replace(1,'Tower',inplace=True)
test_data['buildingType'].replace(2,'Bungalow',inplace=True)
test_data['buildingType'].replace(3,'Tower and Plate',inplace=True)
test_data['buildingType'].replace(4,'Plate',inplace=True)



C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19436\1882406110.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['constructionTime'].replace({'未知': 0}, inplace=True)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19436\1882406110.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a

In [40]:
train_data.columns

Index(['ID', 'Lng', 'Lat', 'tradeTime', 'followers', 'square', 'livingRoom',
       'drawingRoom', 'kitchen', 'bathRoom', 'floor', 'buildingType',
       'constructionTime', 'renovationCondition', 'buildingStructure',
       'ladderRatio', 'elevator', 'fiveYearsProperty', 'subway', 'district',
       'communityAverage', 'distance', 'building_age'],
      dtype='object')

In [41]:
#lấy những columns cần thiết để training
train = ['Lng','Lat','followers','square', 'livingRoom',
       'drawingRoom', 'kitchen', 'bathRoom', 'floor', 'buildingType',
       'constructionTime', 'renovationCondition', 'buildingStructure','ladderRatio', 'elevator', 'fiveYearsProperty', 'subway', 'district',
       'communityAverage','distance','building_age']
train_data = train_data[train]
test_data = test_data[train]
#obtain all the numerical features
numerical_features = train_data.dtypes[train_data.dtypes != 'object'].index
#apply standardization to each feature
train_data[numerical_features] = train_data[numerical_features].apply( lambda x: (x-x.mean()) / x.std() )
#replace missing values with 0
train_data[numerical_features] = train_data[numerical_features].fillna(0)
#one-hot encoding consider missing values as a category.
train_data = pd.get_dummies(train_data, dummy_na=True)
train_data.shape

# all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))

# #obtain all the numerical features
# numerical_features = all_features.dtypes[all_features.dtypes != 'object'].index
# #apply standardization to each feature
# all_features[numerical_features] = all_features[numerical_features].apply( lambda x: (x-x.mean()) / x.std() )
# #replace missing values with 0
# all_features[numerical_features] = all_features[numerical_features].fillna(0)
# #one-hot encoding consider missing values as a category.
# all_features = pd.get_dummies(all_features, dummy_na=True)
# all_features.shape

# pd.isna(all_features).any()

(255080, 133)

In [42]:
train_data.shape

(255080, 133)

In [43]:
#Tách dữ liệu thành tập huấn luyện và tập kiểm tra
# n_train = train_data.shape[0]
# train_features = all_features[:n_train]
# test_features = all_features[n_train:]
# train_labels = Y_train.iloc[:, 1].values

# Standardized the Features
X = np.asarray(train_data)
X = preprocessing.StandardScaler().fit(X).transform(X)  

# Selecting Target Feature
y =  np.asarray(Y_train['TARGET'])#.reshape(-1, 1)

# Split the data into training and test data to see results on the test data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.250001, random_state=42)

# Further spliting the Traning set into sub Train and Test Data to see results on training data
train_X,test_X,train_y,test_y = train_test_split(X_train,y_train,random_state=42)

# Tách tập huấn luyện thành tập train và validation
# x_train, x_valid,y_train, y_valid = train_test_split(train_features, train_labels, test_size=0.250001, random_state = 27)


In [44]:
# print(train_features.shape)  # Số lượng mẫu trong features
# print(train_labels.shape)
# print(Y_train.shape)# Số lượng mẫu trong labels


In [45]:
# Hybrid Model Training For Test Data
# Model 1
r1 = xgb.XGBRegressor(objective ='reg:squarederror', min_child_weight = 2, subsample = 1,
                          colsample_bytree = 0.8,
                          learning_rate = 0.2, n_estimators = 500,
                         reg_lambda = 0.45, reg_alpha = 0, gamma = 0.5)
# Model 2
r2 = lgb.LGBMRegressor(objective='regression',num_leaves=36,learning_rate=0.15,
                        n_estimators=64,min_child_weight = 2, colsample_bytree = 0.8,
                        reg_lambda = 0.45)
# Model 3
r3 = RandomForestRegressor(random_state=42,n_estimators=900,max_depth=20,
                                              n_jobs=-1,min_samples_split=10,)

# Making Hybrid Model
vreg = VotingRegressor([('gb', r1), ('lgb', r2), ('rf', r3)])

# Fitting the Model
vreg.fit(X_train, y_train)



# Evaluation

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035022 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2062
[LightGBM] [Info] Number of data points in the train set: 191309, number of used features: 80
[LightGBM] [Info] Start training from score 348.989451


KeyboardInterrupt: 

In [36]:
print(X_test.shape)

(63770, 133)


In [35]:
# Predicting the Model
pred_y = vreg.predict(X_test)

# Step 4: Create a DataFrame with ID and TARGET results
results_df = pd.DataFrame({
    'ID': range(len(pred_y)),  # Creating ID column from index
    'TARGET': y_pred           # Using the predictions directly
})

# Step 5: Export to CSV
results_df.to_csv('predictions.csv', index=False)

print("Predictions have been saved to 'predictions.csv'")


Predictions have been saved to 'predictions.csv'
